# pyphysio tutorial

## 2. Algorithms

In this second tutorial we will see how to use the class `Algorithm` to create signal processing pipelines.


A signal processing step is a computational function $F$ that operates on input data (a signal) to produce a result. It is characterized by a set of parameters **p** which regulate its behavior.

![algorithm](img/algorithm.png)

*Figure 1: Abstract representation of a processing step.*

1. In pyphysio each processing step is represented by an instance of a class derived from the generic class `Algorithm`.

2. The type of function or algorithm is given by the class name (e.g. `BeatFromECG` extracts the heartbeats from an ECG signal, `PeakDetection` detects the peaks in the input signal).

3. The parameters of the function/algorithm are the attributes of the created instance.

Therefore, a processing step is defined by creating a new instance of the Class, which is initialized with the given parameters:
```
processing_step = ph.BeatFromECG(parameters)
```
To execute the processing step we need to give as input an instance of the class `Signal`:
```
output = processing_step(input)
```


Algorithms in pyphysio are grouped in four categories (see also the tutorial '3-pipelines'):

* Filters : deterministic algorithms that modify the values of the input signal without changing its nature;
* Estimators : algorithms that aim at extracting information from the input signal which is given in output as a signal with a different nature;
* Indicators : algorithms that operate on the signal to provide a scalar value (or metrics)
* Tools : algorithms that can be useful for the signal processing and return as output one or more numpy arrays or scalars.

In [ ]:
# import packages
from __future__ import division
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

# import data from included examples
from pyphysio import TestData, EvenlySignal

ecg_data = TestData.ecg()
eda_data = TestData.eda()

# create two signals
fsamp = 2048
tstart_ecg = 15
tstart_eda = 5

ecg = EvenlySignal(values = ecg_data, 
                   sampling_freq = fsamp, 
                   signal_nature = 'ecg', 
                   start_time = tstart_ecg)

eda = EvenlySignal(values = eda_data, 
                   sampling_freq = fsamp, 
                   signal_nature = 'eda', 
                   start_time = tstart_eda)

### 2.1 Filters

Filters return a signal which has the same **`signal_nature`** of the input signal. 

The name *`Filters`* recalls the aim of this algorithms which is in general to increase the Signal/Noise ratio by filtering out the unwanted components in a signal (e.g high frequency noise).

In [ ]:
# create a Filter
import pyphysio.filters.Filters as flt

lowpass_50 = flt.IIRFilter(fp=50, fs=75, ftype='ellip')

In [ ]:
# help inline
#?flt.IIRFilter

In [ ]:
# check parameters
print(lowpass_50)
# OR
print(lowpass_50.get())

In [ ]:
# apply a Filter
ecg_filtered = lowpass_50(ecg)

In [ ]:
#plot
ecg.plot()
ecg_filtered.plot()

In [ ]:
# check output type
ecg.get_signal_nature()

### 2.2 Estimators
Estimators are algorithms which aim at extracting the information of interest from the input signal, thus returning a new signal which has a different **`signal_nature`**. 

The name *`Estimators`* recalls the fact that the information extraction depends on the value of the algorithm parameters which might not be known *a-priori*. Thus the result should be considered as an estimate of the real content of information of the input signal.

In [ ]:
# create an Estimator
import pyphysio.estimators.Estimators as est

ibi_ecg = est.BeatFromECG()

In [ ]:
# check parameters
ibi_ecg

In [ ]:
# apply an Estimator
ibi = ibi_ecg(ecg_filtered)

In [ ]:
# plot
ax1 = plt.subplot(211)
ecg.plot()

plt.subplot(212, sharex=ax1)
ibi.plot()

In [ ]:
# check output type
ibi.get_signal_nature()

### 2.3 Indicators

Indicators are algorithm which extract a metrics (scalar value) from the input signal, for instance a statistic (average).

Three types of indicators are provided in **`pyphysio`**:
* Time domain indicators: comprising simple statistical indicators and other metrics that can be computed on the signal values;
* Frequency domain indicators: metrics that are computed on the Power Spectrum Density (PSD) of the signal;
* Non-linear indicators: complex indicators that are computed on the signal values (e.g. Entropy).

In [ ]:
# create an Indicator
import pyphysio.indicators.TimeDomain as td_ind
import pyphysio.indicators.FrequencyDomain as fd_ind

In [ ]:
rmssd = td_ind.RMSSD()
HF = fd_ind.PowerInBand(interp_freq=4, freq_max=0.4, freq_min=0.15, method = 'ar')

In [ ]:
# check parameters
print(rmssd)
print(HF)

In [ ]:
# apply an Indicator
rmssd_ = rmssd(ibi)
HF_ = HF(ibi)

print(rmssd_)
print(HF_)

In [ ]:
# check output type
print(type(rmssd_))
print(type(HF_))

### 2.4 Tools

This is a collection of useful algorithms that can be used for signal processing. 

These algorithms might return scalar values or numpy arrays.

In [ ]:
# create a Tool
import pyphysio.tools.Tools as tll

compute_psd = tll.PSD(method='ar', interp_freq = 4)

In [ ]:
# check parameters
compute_psd

In [ ]:
# apply a Tool
frequencies, power = compute_psd(ibi)

plt.plot(frequencies, power)
plt.show()